In [40]:
import pandas as pd

train_data= pd.read_csv('Groceries data train.csv')
test_data= pd.read_csv('Groceries data test.csv')
print(train_data.head())
print(test_data.head())


   Member_number        Date  itemDescription  year  month  day  day_of_week
0           3021  30/01/2015      frankfurter  2015      1   30            4
1           1292  24/10/2015             pork  2015     10   24            5
2           4206   4/04/2014  root vegetables  2014      4    4            4
3           4369  25/08/2015           onions  2015      8   25            1
4           1522   1/07/2014          waffles  2014      7    1            1
   Member_number        Date itemDescription  year  month  day  day_of_week
0           3481   8/03/2015           candy  2015      3    8            6
1           1254  19/04/2015      white wine  2015      4   19            6
2           2835  28/01/2014   domestic eggs  2014      1   28            1
3           2854   2/08/2015          coffee  2015      8    2            6
4           4637  12/08/2014   bottled water  2014      8   12            1


In [72]:
#count the occurence of items, make it into vectors for later similarity calculation
user_item_matrix= train_data.pivot_table(index='Member_number', columns='itemDescription', aggfunc='size', fill_value=0)
user_item_matrix_test= test_data.pivot_table(index='Member_number', columns='itemDescription', aggfunc='size', fill_value=0)
print(user_item_matrix)

itemDescription  Instant food products  UHT-milk  abrasive cleaner  \
Member_number                                                        
1000                                 0         0                 0   
1001                                 0         0                 0   
1002                                 0         0                 0   
1003                                 0         0                 0   
1004                                 0         0                 0   
...                                ...       ...               ...   
4996                                 0         0                 0   
4997                                 0         0                 0   
4998                                 0         0                 0   
4999                                 0         0                 0   
5000                                 0         0                 0   

itemDescription  artif. sweetener  baby cosmetics  bags  baking powder  \
Member_number  

In [73]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
#build up user similarity matrix and item similarity matrix
user_similarity= cosine_similarity(user_item_matrix)
#take item, which is the transpose of the matrix
item_similarity= cosine_similarity(user_item_matrix.T)

user_similarity_test= cosine_similarity(user_item_matrix_test)

item_similarity_test= cosine_similarity(user_item_matrix_test.T)

print(user_similarity.shape)
print(item_similarity.shape)

print(user_similarity,'\n')

print(item_similarity)


(3872, 3872)
(167, 167)
[[1.         0.32659863 0.         ... 0.         0.         0.15811388]
 [0.32659863 1.         0.21081851 ... 0.36514837 0.         0.25819889]
 [0.         0.21081851 1.         ... 0.         0.21821789 0.        ]
 ...
 [0.         0.36514837 0.         ... 1.         0.         0.        ]
 [0.         0.         0.21821789 ... 0.         1.         0.40089186]
 [0.15811388 0.25819889 0.         ... 0.         0.40089186 1.        ]] 

[[1.         0.03106849 0.         ... 0.06684553 0.05292561 0.        ]
 [0.03106849 1.         0.01937461 ... 0.15462642 0.10551045 0.03106849]
 [0.         0.01937461 1.         ... 0.02273755 0.01650246 0.        ]
 ...
 [0.06684553 0.15462642 0.02273755 ... 1.         0.30336983 0.10634516]
 [0.05292561 0.10551045 0.01650246 ... 0.30336983 1.         0.03969421]
 [0.         0.03106849 0.         ... 0.10634516 0.03969421 1.        ]]


In [43]:
def recommendation(user_id, user_similarity, item_similarity, user_item_matrix, num=5):
    #find location in the user_item_matrix
    line= user_item_matrix.index.get_loc(user_id)
    #find the certain user's similarity with other users 
    similar_users= user_similarity[line]
    
    user_ratings= user_item_matrix.iloc[line]
    #calculate the weighted average of the ratings
    recommendations= user_ratings.dot(item_similarity) / similar_users.sum()
    
    #sort by the predicted rating in descending order
    recommendation_idx= np.argsort(recommendations)[::-1]
    #select the top (num), return column index
    recommendation_idx=recommendation_idx[:num]

    return user_item_matrix.columns[recommendation_idx]

In [44]:
for i in range(len(user_item_matrix)):
    user_id= user_item_matrix.index[i]
    recommendations= recommendation(user_id, user_similarity, item_similarity, user_item_matrix, 5)
    print(f"User{user_id}: {list(recommendations)}")

User1000: ['sausage', 'soda', 'yogurt', 'canned beer', 'pastry']
User1001: ['whole milk', 'soda', 'rolls/buns', 'sausage', 'whipped/sour cream']
User1002: ['whole milk', 'tropical fruit', 'butter', 'frozen vegetables', 'sugar']
User1003: ['rolls/buns', 'root vegetables', 'sausage', 'detergent', 'dental care']
User1004: ['whole milk', 'rolls/buns', 'other vegetables', 'pip fruit', 'canned beer']
User1005: ['rolls/buns', 'whipped/sour cream', 'margarine', 'whole milk', 'other vegetables']
User1006: ['frankfurter', 'rolls/buns', 'bottled beer', 'shopping bags', 'chicken']
User1008: ['yogurt', 'tropical fruit', 'domestic eggs', 'hamburger meat', 'dessert']
User1009: ['tropical fruit', 'yogurt', 'pastry', 'herbs', 'canned fish']
User1010: ['bottled water', 'pip fruit', 'coffee', 'UHT-milk', 'specialty bar']
User1011: ['whole milk', 'other vegetables', 'rolls/buns', 'bottled water', 'pastry']
User1012: ['whole milk', 'rolls/buns', 'yogurt', 'shopping bags', 'tropical fruit']
User1013: ['whol

In [69]:
#code from task 1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
def task_1(data):


    data.fillna(0, inplace=True)

    grouped_data = data.groupby(0)[2].apply(lambda x: list(set(x)))

    transactions = grouped_data.tolist()

    from apyori import apriori
    from mlxtend.frequent_patterns import association_rules
    data.fillna(0, inplace=True)
    grouped_data = data.groupby(0)[2].apply(lambda x: list(set(x)))

    transactions = grouped_data.tolist()

    rules = apriori(transactions, min_support=0.003, min_confidence=0.2, min_lift=3, min_length=2)
    results = list(rules)
    df_result = pd.DataFrame(results)
    support = df_result.support
    first_values = []
    second_values = []
    third_values = []
    fourth_values = []
    print(df_result.shape[1])
    for i in range(df_result.shape[0]):
        single_list = df_result['ordered_statistics'][i][0]
        first_values.extend(list(single_list[0]))
        second_values.extend(list(single_list[1]))
        third_values.append(single_list[2])
        fourth_values.append(single_list[3])
    lhs = pd.DataFrame(first_values)
    rhs = pd.DataFrame(second_values)
    confidance = pd.DataFrame(third_values, columns=['Confidence'])

    lift = pd.DataFrame(fourth_values, columns=['Lift'])
    df_final = pd.concat([lhs, rhs, support, confidance, lift], axis=1)
    df_final
    df_final.fillna(value=' ', inplace=True)
    df_final.head()
    df_final.columns = ['lhs','rhs', 'support', 'confidence', 'lift']
    df_final.head()

    return df_final
df_final=task_1(pd.read_csv('Groceries data train.csv', header=None))
df_final_test=task_1(pd.read_csv('Groceries data test.csv', header=None))

3
3


In [70]:
def combined_recommendation(user_id, user_similarity, item_similarity, user_item_matrix, freq_patterns, num=5):
    # collaborative filtering recommendations
    cf_recommendations = recommendation(user_id, user_similarity, item_similarity, user_item_matrix, num//2)
    
    # frequent pattern recommendations
    pattern_recommendations = []
    for pattern in freq_patterns.itertuples():
        if set(pattern.lhs.split(', ')).issubset(user_item_matrix.columns):
            pattern_recommendations.extend(pattern.rhs.split(', '))
    while ' ' in pattern_recommendations:
        pattern_recommendations.remove(' ')
        
    # combine recommendations
    #print(cf_recommendations)
    combined_recommendations = list(cf_recommendations)+list(pattern_recommendations)
    
    return combined_recommendations[:num]

for i in range(len(user_item_matrix)):
    user_id = user_item_matrix.index[i]
    recommendations = combined_recommendation(user_id, user_similarity, item_similarity, user_item_matrix, df_final, 5)
    print(f"User {user_id}: {list(recommendations)}")

User 1000: ['sausage', 'soda', 'shopping bags', 'bottled beer', 'butter']
User 1001: ['whole milk', 'soda', 'shopping bags', 'bottled beer', 'butter']
User 1002: ['whole milk', 'tropical fruit', 'shopping bags', 'bottled beer', 'butter']
User 1003: ['rolls/buns', 'root vegetables', 'shopping bags', 'bottled beer', 'butter']
User 1004: ['whole milk', 'rolls/buns', 'shopping bags', 'bottled beer', 'butter']
User 1005: ['rolls/buns', 'whipped/sour cream', 'shopping bags', 'bottled beer', 'butter']
User 1006: ['frankfurter', 'rolls/buns', 'shopping bags', 'bottled beer', 'butter']
User 1008: ['yogurt', 'tropical fruit', 'shopping bags', 'bottled beer', 'butter']
User 1009: ['tropical fruit', 'yogurt', 'shopping bags', 'bottled beer', 'butter']
User 1010: ['bottled water', 'pip fruit', 'shopping bags', 'bottled beer', 'butter']
User 1011: ['whole milk', 'other vegetables', 'shopping bags', 'bottled beer', 'butter']
User 1012: ['whole milk', 'rolls/buns', 'shopping bags', 'bottled beer', 'bu

In [74]:
train_recommendations = {}
for user_id in user_item_matrix.index:
    recommendations = combined_recommendation(user_id, user_similarity, item_similarity, user_item_matrix, df_final, 5)
    train_recommendations[user_id] = recommendations
test_recommendations = {}
for user_id in user_item_matrix_test.index:
    recommendations_test = combined_recommendation(user_id, user_similarity_test, item_similarity_test, user_item_matrix_test, df_final_test, 5)
    test_recommendations[user_id] = recommendations_test
    
def evaluate_recommendations(test_data, recommendations, user_item_matrix):
    precisions = []
    recalls = []
    f1_scores = []
    
    for user_id in test_data.index:
        if user_id not in recommendations:
            continue
        #find actual purchase record
        actual_items = set(user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] > 0].index)
        #produce recommendation
        recommended_items = set(recommendations[user_id])
        #calculate the difference
        true_positives = actual_items.intersection(recommended_items)
        false_positives = recommended_items - actual_items
        false_negatives = actual_items - recommended_items
        #calculate precision, recall and f1
        precision = len(true_positives) / (len(true_positives) + len(false_positives)) if (len(true_positives) + len(false_positives)) > 0 else 0
        recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if (len(true_positives) + len(false_negatives)) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    #take the mean of all difference measurements
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    
    return avg_precision, avg_recall, avg_f1_score


precision, recall, f1 = evaluate_recommendations(train_data, train_recommendations, user_item_matrix)
precision_test, recall_test, f1_test = evaluate_recommendations(test_data, test_recommendations, user_item_matrix_test)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

print(f"Test_Precision: {precision_test:.2f}")
print(f"Test_Recall: {recall_test:.2f}")
print(f"Test_F1 Score: {f1_test:.2f}")

Precision: 0.45
Recall: 0.45
F1 Score: 0.41
Test_Precision: 0.64
Test_Recall: 0.72
Test_F1 Score: 0.62
